In [ ]:
import math
import sys

import numpy as np
from scipy.ndimage import gaussian_filter
from scipy.stats import multivariate_normal


import torch
import torch.nn as nn

import matplotlib.pyplot as plt

sys.path.append('..')

from src.features.dihedral import DihedralLayer

from dihedral import cal_dihedral

coords_path = "../../data/NPY/c_trj.npy"
forces_path = "../../data/NPY/f_trj.npy"

In [ ]:
coords = np.load(coords_path)
forces = np.load(forces_path)

In [ ]:
dihedral = DihedralLayer(coords.shape[1])(torch.tensor(coords), is_sin_cos=False)
dihedral = dihedral.numpy()

In [ ]:
psi = dihedral[::, 0::2].reshape(-1)
phi = dihedral[::, 1::2].reshape(-1)

In [ ]:
print(psi.shape)
print(phi.shape)

In [ ]:
plt.scatter(np.rad2deg(phi[0:100000])+179, np.rad2deg(psi[0:100000]) + 179)
plt.show()

In [ ]:
grid = 50
delta = 360 / grid

cmap = np.zeros((grid, grid))
psi_deg = np.rad2deg(psi) + 179
phi_deg = np.rad2deg(phi) + 179

for i in range(psi_deg.shape[0]):
    phi_grid = int(phi_deg[i] / delta)
    psi_grid = int(psi_deg[i] / delta)
    cmap[psi_grid, phi_grid] += 1

In [ ]:
def plot_cmap_heat(grid, cmap):
    fig, ax = plt.subplots()
    # heatmap = ax.pcolor(cmap, cmap=plt.cm.Blues)
    cmap_ = np.copy(cmap)
    cmap_[cmap_ > 1] = np.log(cmap_[cmap_ > 1])
#     heatmap = ax.pcolor(cmap, cmap=plt.cm.Blues)
    heatmap = ax.pcolor(cmap_)
    plt.show()

In [ ]:
plot_cmap_heat(grid, cmap)

In [ ]:
filterd_cmap = gaussian_filter(cmap, sigma=2, truncate=4)
# filterd_cmap = gaussian_filter(filterd_cmap, sigma=1, truncate=10)
plot_cmap_heat(grid, filterd_cmap)

filterd_cmap_ = np.copy(filterd_cmap)
filterd_cmap_[filterd_cmap_ == 0] = 100
plot_cmap_heat(grid, filterd_cmap_)

In [ ]:
filterd_cmap[filterd_cmap == 0]

In [ ]:
m = 2 #dimension
mean = np.zeros(m)
sigma = np.eye(m)

N = grid


def plot_energy_surface(cmap_, epsilon):
    cmap_ = cmap_ + epsilon
    energy = -1 * np.log(cmap_)
    energy = energy + np.abs(np.min(energy))
    
    x1 = np.linspace(0, 50, N)
    x2 = np.linspace(0, 50, N)

    X1, X2 = np.meshgrid(x1, x2)
    X = np.c_[np.ravel(X1), np.ravel(X2)]

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.view_init(elev=50, azim=45)
    surf = ax.plot_surface(X1, X2, energy, cmap='bwr', linewidth=0)
    fig.colorbar(surf)
#     ax.set_title("Surface Plot")
    fig.show()
    plot_cmap_heat(grid, energy)

In [ ]:
plot_energy_surface(filterd_cmap, 1e-4)

In [ ]:
plot_energy_surface(cmap, 1e-4)

In [ ]:
# Forceの可視化

def cal_energy(cmap_, epsilon=1e-4):
    cmap_ = cmap_ + epsilon
    energy = -1 * np.log(cmap_)
    energy = energy + np.abs(np.min(energy)) + epsilon
    return energy

def cal_force(energy):
    force_x = np.zeros_like(energy)
    force_y = np.zeros_like(energy)
    
    for i in range(grid):
        for j in range(grid):
            x_max = 0 if i == grid - 1 else i + 1
            x_min = i - 1
            
            y_max = 0 if j == grid - 1 else j + 1
            y_min = j - 1
            
#             print(x_max, x_min, y_max, y_min)
            
            force_x[i,j] = -1 * (energy[x_max, j] - energy[x_min, j]) / delta
            force_y[i,j] = -1 * (energy[i, y_max] - energy[i, y_min]) / delta
            
    return force_x, force_y

In [ ]:
def cmap_forces(energy, log=False):
    cmap_force_x, cmap_force_y = cal_force(energy)
    x1 = np.linspace(0, N, N)
    x2 = np.linspace(0, N, N)
    
    xx, yy = np.meshgrid(x1, x2)
    if log:
        cmap_force_x_sign = np.sign(cmap_force_x)
        cmap_force_y_sign = np.sign(cmap_force_y)
        cmap_force_x = np.log(np.abs(cmap_force_x)) * cmap_force_x_sign
        cmap_force_y = np.log(np.abs(cmap_force_y)) * cmap_force_y_sign
    plt.quiver(xx, yy, cmap_force_y, cmap_force_x)
    plt.figure(dpi=1, figsize=(10, 10))
    plt.show()

In [ ]:
# cmap_force_x, cmap_force_y = cal_force(cmap)

# x1 = np.linspace(0, 50, N)
# x2 = np.linspace(0, 50, N)

# xx, yy = np.meshgrid(x1, x2)

# plt.quiver(xx, yy, cmap_force_x, cmap_force_y)
# plt.show()
cmap_energy = cal_energy(cmap)
filterd_energy = cal_energy(filterd_cmap)

cmap_forces(cmap_energy)
cmap_forces(filterd_energy)